# 6. Classification sur le dataset MNIST

## 6.0 Régression logistique et fonction sigmoïde

Le modèle de régression logistique est la composée d'une transformation linéaire du vecteur des observations et de la fonction sigmoïde.

$$f_{\mathbf{w},b} = \frac{e^{\mathbf{wx}+b}}{1 + e^{\mathbf{wx}+b}}$$

In [ ]:
# %run miscellaneous.ipynb
# https://filesender.renater.fr/?s=download&token=479c4058-ca6a-464f-ac46-4bdcf7f5ddae

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
def sigmoid(x):
    return np.exp(x) / (np.exp(0) + np.exp(x))

x = np.linspace(-6, 6, 100)
S = sigmoid(x)
plt.plot(x, S, color='blue', lw=2)
plt.xlabel("$x$")
plt.ylabel("$sigmoid(x)$");

## 6.1 Jeu de données MNIST

Nous chargeons le jeu de données MNIST composé de $70 000$ images de chiffres en nuances de gris de 28 pixels par 28 pixels. La valeur de chaque pixel est comprise entre $0$ et $255$ et qualifie une nuance de gris. L'objectif est de prédire la valeur du chiffre ($0$ à $9$).

In [ ]:
from sklearn.datasets import fetch_openml

#X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

import pandas as pd
X = pd.read_csv('/data/mnist_784_X.csv')
y = pd.read_csv('/data/mnist_784_y.csv')

In [ ]:
print("Dimensions de X : ", X.shape, " ; dimensions de y : ", y.shape)

In [ ]:
X

Nous créons une fonction `print_digit` pour afficher un chiffre à partir de son vecteur. Vous pouvez l'utiliser en lui donnant comme paramètre un tableau numpy. Utilisez la pour afficher quelques images de notre dataset (l'attribut `.values` d'une series pandas est un tableau numpy), pensez à afficher aussi le chiffre correspondant (i.e. : la classe) du vecteur `y` :

In [ ]:
def print_digit(vec):
    img = vec.values.reshape(28,28)
    plt.imshow(img, cmap = "gray")
    plt.axis("off")
    plt.show()

In [ ]:
import random
sample_id = random.randint(0, len(X))
print("X[0] :")
print_digit(X.iloc[sample_id])
print("y[0] : ", y.iloc[sample_id])

Les labels sont représentés par des chaînes de caractères. Pour un traitement plus facile, transformez les en nombres entiers en utilisant la méthode [`astype`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html) de la series `y` :

In [ ]:
y = y.astype(int)  # or np.uint8

Découpez votre dataset en un dataset d'entrainement et un dataset de test. Pour réduire le temps de calcul des entrainements suivants, vous pouvez choisir de ne prendre qu'une partie des données pour l'entrainement (30 000 images par exemple) en fournissant un paramètre entier à la fonction [`sklearn.model_selection.train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) :

In [ ]:
train_size = 30000
test_size = 10000

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=train_size,test_size=test_size)

## 6.2 Classification binaire

Pour traiter le cas de la classification binaire, nous considérons le problème de la détection du chiffre $8$. Créez deux series `y_train_8` et `y_test_8` contenant des booléens indiquant si le chiffre est un $8$ ou non :

In [ ]:
y_train_8 = (y_train == 8)
y_test_8 = (y_test == 8)

Entrainez un modèle de [`sklearn.linear_model.LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) en recherchant les meilleurs hyper-paramètres par validation croisée avec 3 plis (pour réduire le temps de calcul). Votre grille d'hyper-paramètres pourra se concentrer sur le coefficient de régularisation `C` que vous pouvez faire varier avec `np.logspace(-5, 5, 10)`. Puis affichez les meilleurs hyper-paramètres trouvés.

Pensez à centrer et réduire vos données dans un pipeline avec la classe [`sklearn.preprocessing.StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).

En effet, certains algorithmes d'optimisation de la fonction de coût se comportent mieux quand les données sont standardisées. C'est en particulier le cas des algorithmes de régularisation comme _ridge_, _LASSO_ ou _elastic net_. Par exemple, __[la documentation de scikit-learn pour l'algorithme de régression logistique](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)__ précise :

>Note that ‘sag’ and ‘saga’ fast convergence is only guaranteed on features with approximately the same scale. You can preprocess the data with a scaler from sklearn.preprocessing.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

In [ ]:
# Alternative 1 pour créer un pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('log_reg', LogisticRegression(solver='saga',
                                   tol=0.1,
                                   penalty='l2'))
])
param_grid = [
    {'log_reg__C': np.logspace(-5, 5, 10)}
]

# Alternative 2 pour créer un pipeline
pipeline = make_pipeline(StandardScaler(), LogisticRegression(solver='saga', tol=0.1))
param_grid = [
    {'logisticregression__C': np.logspace(-5, 5, 10)}
]


grid_search = GridSearchCV(
    pipeline, param_grid, cv=3, scoring="accuracy", verbose=3, n_jobs=10
)
grid_search.fit(X_train, y_train_8)

print("Meilleurs hyper-paramètres : ", grid_search.best_params_)

Calculez l'exactitude (`accuracy`) d'un modèle entrainé avec ces hyper-paramètres sur le jeu d'entrainement avec une validation croisée à 10 plis :

In [ ]:
scores = cross_val_score(grid_search.best_estimator_, X_train, y_train_8, scoring="accuracy", cv=10)
np.mean(scores), np.std(scores)

Calculez la proportion de valeur `True` dans `y_train_8` (donc la proportion de chiffre 8 dans notre dataset d'entrainement). Ce dataset est-il équilibré ? Est-ce que l'exactitude suffit à mesurer la performance d'un modèle sur ce dataset ? Pouvez-vous imaginer un modèle simple pour ce dataset ayant des performances proches du modèle entrainé avec une regression logistique ?

In [ ]:
1 - (sum(y_train_8.values)/len(y_train_8))

Le jeu de données est déséquilibré, moins de 10\% des observations sont des chiffres "$8$". Dans ce contexte, l'exactitude n'est pas la mesure de performance la mieux adaptée. Un modèle répondant qu'une image ne correspond jamais à un $8$ aurait une exactitude d'environ 90%.

## 6.3 Mesures de performance

### 6.3.1 Matrice de confusion

Utilisez la fonction [`sklearn.model_selection.cross_val_predict`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) (avec 10 plis) pour récupérer les labels prédits (et non seulement un score) sur une validation croisée. Stockez le résultat dans une variable `y_train_8_pred`, ces résultats nous servirons à mesurer les performances de notre modèle :

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

y_train_8_pred = cross_val_predict(
    grid_search.best_estimator_,
    X_train, y_train_8, cv=10
)

In [ ]:
y_train_8_pred

Utilisez la fonction [`sklearn.metrics.confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) pour calculer la matrice de confusion sur ces prédictions :

In [ ]:
confusion_matrix(y_train_8, y_train_8_pred)

L'affichage obtenu peut-être amélioré avec un graphique, la fonction suivante permet d'afficher un tel graphique. Elle prend en paramètre les trois listes suivantes :
* `y_true` : les vérités terrains
* `y_pred` : les prédictions
* `labels` : les labels à afficher (dans notre cas, vous pouvez utiliser `['non_8', '8']`)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred)
    ax= plt.subplot()
    #annot=True to annotate cells, ftm='g' to disable scientific notation
    sns.heatmap(cm, annot=True, fmt='g', ax=ax)
    ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix')
    ax.xaxis.set_ticklabels(labels)
    ax.yaxis.set_ticklabels(labels)

In [ ]:
plot_confusion_matrix(y_train_8, y_train_8_pred, ['non_8', '8'])

### 6.3.2 Précision, Rappel, score F1

Calculez et affichez la précision, le rappel et le score F1 avec les fonctions [`sklearn.metrics.precision_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html), [`sklearn.metrics.recall_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html) et [`sklearn.metrics.f1_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html), puis concluez sur les performances du modèle :

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
print("Précision : ", precision_score(y_train_8, y_train_8_pred))
print("Rappel : ", recall_score(y_train_8, y_train_8_pred))
print("F1 : ", f1_score(y_train_8, y_train_8_pred))

### 6.3.3 Courbe précision / rappel

La plupart des modèles de classification retournent une probabilité d'appartenance à une classe (dans le cas binaire, une probabilité que l'individu soit de la classe $1$, ou que le chiffre soit un $8$ dans notre cas). Les métriques précédentes sont calculées pour un seuil de décision de $0.5$.

Affichez une image du dataset et calculez la probabilité que ce soit un $8$ pour notre modèle en utilisant la fonction `predict_proba` :

In [ ]:
sample_id = random.randint(0, len(X_train))
print_digit(X_train.iloc[sample_id])
print(
    "Probabilités associées aux deux classes : ",
    grid_search.best_estimator_.predict_proba([X_train.iloc[sample_id]])
)

Le score $F_1$ privilégie les modèles avec une précision et un rappel semblables. Selon le contexte, nous pouvons préférer un modèle avec une bonne précision ou un bon rappel. Ainsi, il est intéressant de contrôler le seuil de probabilité utilisé pour prendre la décision.

Pour cela, nous avons besoin de récupérer les probabilités associées à chaque individu du jeu d'entrainement. Utilisez la fonction [`sklearn.model_selection.cross_val_predict`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) avec 10 plis et le paramètre `method="predict_proba"` pour récupérer ces probabilités. Stockez le résultat dans une variable `y_train_8_scores`. Vous obtiendrez un tableau des probabilités pour les deux classes (non 8 et 8), transformez ce tableau en un tableau de probabilités pour la classe "est un 8" `np.array([y[1] for y in y_train_8_scores])` :

In [ ]:
y_train_8 = y_train_8['class']

In [ ]:
y_train_8_scores = cross_val_predict(
    grid_search.best_estimator_,
    X_train,
    y_train_8,
    cv=10,
    method="predict_proba"
)
y_train_8_scores

In [ ]:
y_train_8_scores = np.array([y[1] for y in y_train_8_scores])
y_train_8_scores

Nous pouvons faire varier le seuil de désision pour obtenir un meilleur rappel au détriment de la précision (et inversement). Calculez d'abord la précision et le rappel pour différents seuils de décision en utilisant la fonction [`sklearn.metrics.precision_recall_curve`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html) :

In [ ]:
from sklearn.metrics import precision_recall_curve
precisions, recalls, probas = precision_recall_curve(
    y_train_8, y_train_8_scores
)

La fonction suivante affiche un graphique des courbes de précision et rappel en fonction des seuils de décisions possibles, ses paramètres sont les mêmes que la sortie de la fonction `precision_recall_curve` :

In [ ]:
def plot_precision_recall_proba(precisions, recalls, probas):
    plt.plot(probas, precisions[:-1], "b--", label="Précision", linewidth=2)
    plt.plot(probas, recalls[:-1], "g-", label="Rappel", linewidth=2)
    plt.xlabel("Seuil de probabilité", fontsize=16)
    plt.legend(loc="best", fontsize=16)
    plt.ylim([0, 1])

In [ ]:
plt.figure(figsize=(8, 4))
plot_precision_recall_proba(precisions, recalls, probas)

Nous pouvons aussi utiliser la fonction suivante pour afficher la précision en fonction du rappel, ce qui nous permet de choisir l'un en fonction de l'autre en fonction de nos objectifs. Ses paramètres sont les précisions et les rappels calculés précédement :

In [ ]:
def plot_precision_recall(precisions, recalls):
    plt.plot(recalls, precisions, "b-", linewidth=2)
    plt.xlabel("Rappel", fontsize=16)
    plt.ylabel("Précision", fontsize=16)
    plt.axis([0, 1, 0, 1])

In [ ]:
plt.figure(figsize=(8, 8))
plot_precision_recall(precisions, recalls)

### 6.3.4 Courbe ROC

La courbe ROC, une autre représentation courante, trace l'évolution du taux des vrais positifs (autrement dit le rappel ou la sensibilité) en fonction du taux de faux positifs (autrement dit $1 - \text{spécificité}$). La ligne diagonale pointillée correspond à un modèle qui prédit aléatoirement.

In [ ]:
def plot_roc(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('Taux de faux positifs', fontsize=16)
    plt.ylabel('Taux de vrais positifs', fontsize=16)

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, probas = roc_curve(y_train_8, y_train_8_scores)

plt.figure(figsize=(8, 8))
plot_roc(fpr, tpr)

## 6.4 Classes multiples

Entrainer un modèle _softmax_ avec régularisation _elasticnet_ sur la classification en classes multiples en recherchant les meilleurs hyper-paramètres puis en calculant les performances de ces hyper-paramètres sur le dataset d'entrainement avec une validation croisée à 10 plis.

Le modèle _softmax_ est disponible via la classe `LogisticRegression` en utilisant la valeur `multinomial` pour le paramètre `multi_class`. Pour une régularisation _elasticnet_, il faut définir le paramètre `penalty` à `l2`. Vous pouvez aussi utiliser les paramètres `solver='saga'` et `tol=0.1` pour que l'algorithme converge plus rapidement.

Calculez l'exactitude de ce modèle et affichez ses prédictions pour quelques invididus de notre jeu d'entrainement :

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('log_reg', LogisticRegression(multi_class='multinomial',
                                   solver='saga',
                                   tol=0.1, penalty='l2'))
])
param_grid = [{'log_reg__C': np.logspace(-5, 5, 5)}]
grid_search = GridSearchCV(pipeline,
                           param_grid, cv=3,
                           scoring="f1_weighted")
grid_search.fit(X_train, y_train)

print("Meilleurs hyper-paramètres : ", grid_search.best_params_)

scores = cross_val_score(grid_search.best_estimator_, X_train,
                         y_train, scoring="accuracy", cv=10)

print("Moyenne et écart type accuracy : %.4f (%.4f)" % (scores.mean(),
                                                        scores.std()))

In [ ]:
sample = X_train.sample().iloc[0]
print_digit(sample)
print(
    "Prédiction:", grid_search.best_estimator_.predict([sample])
)
print(
    "Probabilités associées à toutes les classes : ",
    grid_search.best_estimator_.predict_proba([sample])
)

### 6.4.1 Visualisation des poids associés à chaque transformation linéaire du modèle softmax

Nous pouvons récupérer poids associés à chaque pixels des images et les afficher pas classe :

In [ ]:
coef = grid_search.best_estimator_.steps[1][1].coef_
plt.figure(figsize=(10, 5))
scale = np.abs(coef).max()
for i in range(10):
    plot = plt.subplot(2,5,i+1)
    plot.imshow(coef[i].reshape(28,28), cmap=plt.cm.Greys, vmin=-scale, vmax=scale)
    plot.set_xticks(())
    plot.set_yticks(())
    plot.set_xlabel('Classe %i' % i)
plt.show()

### 6.4.2 Visualisation des erreurs

La matrice de confusion est toujours utile à analyser. Pour ce problème, vu le nombre important de classes, nous pouvons afficher une version simplifiée de cette matrice :

In [ ]:
y_train_pred = cross_val_predict(grid_search.best_estimator_, X_train, y_train, cv=3)

In [ ]:
confusion = confusion_matrix(y_train, y_train_pred)
plt.matshow(confusion, cmap=plt.cm.gray)
plt.show()

Pour y voir plus clair, nous remplaçons la diagonale (les prédictions correctes) par des 0. Nous mettons donc en avant uniqument les cas d'erreur :

In [ ]:
np.fill_diagonal(confusion, 0)
plt.matshow(confusion, cmap=plt.cm.gray);

Nous pouvons aussi afficher aléatoirement un cas d'erreur de notre prédicteur :

In [ ]:
y_train_with_pred = y_train.copy()
y_train_with_pred['pred'] = y_train_pred
sample = y_train_with_pred[y_train_with_pred["class"] != y_train_with_pred.pred].sample().iloc[0]
print(sample.name)
print_digit(X_train.loc[sample.name])
print(
    "Truth:", sample['class'],
    "Prediction:", sample['pred']
)

## 6.5 Forêt aléatoire

Entrainez un modèle de forêt aléatoire sur le problème MNIST puis comparez le modèle obtenu avec le modèle _sortmax_.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(n_jobs=-1,
                                  n_estimators=100,
                                  random_state=77))
])
param_grid = {
    'rf__max_depth': np.linspace(10,50,5, dtype=int),
    'rf__min_samples_leaf': [2,4,8]
}
grid_search = GridSearchCV(rf_pipeline,
                           param_grid, cv=3,
                           scoring="f1_weighted")
grid_search.fit(X_train, y_train)

print("Meilleurs hyper-paramètres : ", grid_search.best_params_)

scores = cross_val_score(grid_search.best_estimator_, X_train,
                         y_train, scoring="accuracy", cv=10)

print("Moyenne et écart type accuracy : %.4f (%.4f)" % (scores.mean(),
                                                        scores.std()))

In [ ]:
y_train_pred_rf = cross_val_predict(
    grid_search.best_estimator_, X_train, y_train, cv=3
)

In [ ]:
confusion_rf = confusion_matrix(y_train, y_train_pred_rf)
np.fill_diagonal(confusion_rf, 0)
plt.matshow(confusion_rf, cmap=plt.cm.gray);

## 6.6 Evaluation du meilleur modèle sur le jeu de test

In [ ]:
from sklearn.metrics import f1_score

y_pred = grid_search.best_estimator_.predict(X_test)

print("F1 : ", f1_score(y_test, y_pred, average='weighted'))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)